## 循环神经网络的简洁实现

虽然 8.5节 对了解循环神经⽹络的实现⽅式具有指导意义，但并不⽅便。本节将展⽰如何使⽤深度学习框架
的⾼级API提供的函数更有效地实现相同的语⾔模型。我们仍然从读取时光机器数据集开始。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [ ]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

### 定义模型

⾼级API提供了循环神经⽹络的实现。我们构造⼀个具有256个隐藏单元的单隐藏层的循环神经⽹络
层rnn_layer。事实上，我们还没有讨论多层循环神经⽹络的意义（这将在 9.3节中介绍）。现在仅需要将
多层理解为⼀层循环神经⽹络的输出被⽤作下⼀层循环神经⽹络的输⼊就⾜够了。

In [ ]:
num_hiddens = 256
rnn_layer = nn.RNN(len(vocab), num_hiddens)

我们使⽤张量来初始化隐状态，它的形状是（隐藏层数，批量⼤⼩，隐藏单元数）。

In [ ]:
state = torch.zeros((1, batch_size, num_hiddens))
state.shape

通过⼀个隐状态和⼀个输⼊，我们就可以⽤更新后的隐状态计算输出。需要强调的是，rnn_layer的“输出” （Y）
不涉及输出层的计算：它是指每个时间步的隐状态，这些隐状态可以⽤作后续输出层的输⼊。

In [ ]:
X = torch.rand(size=(num_steps, batch_size, len(vocab)))
Y, state_new = rnn_layer(X, state)
Y.shape, state_new.shape

与 8.5节类似，我们为⼀个完整的循环神经⽹络模型定义了⼀个RNNModel类。注意，rnn_layer只包含隐藏的
循环层，我们还需要创建⼀个单独的输出层。

In [2]:
class RNNModel(nn.Module):
    """循环神经网络模型"""
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        # 如果RNN是双向的（之后将介绍），num_directions应该是2，否则应该是1
        if not self.rnn.bidirectional:
          self.num_directions = 1
          self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
          self.num_directions = 2
          self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 全连接层首先将Y的形状改为（时间步数*批量大小，隐藏层单元数）
        # 它的输出形状是（时间步数*批量大小，词表大小）。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
          # nn.GRU以张量作为隐状态
          return torch.zeros((self.num_directions * self.rnn.num_layers,
                     batch_size, self.num_hiddens),
                     device=device)
        else:
          # nn.LSTM以元组作为隐状态
          return (torch.zeros((
              self.num_directions*self.rnn.num_layers,
              batch_size, self.num_hiddens), device=device),
                torch.zeros((
                    self.num_directions*self.rnn.num_layers,
                    batch_size, self.num_hiddens),device=device))

### 训练与预测

在训练模型之前，让我们基于⼀个具有随机权重的模型进⾏预测。

In [ ]:
device = d2l.try_gpu()
net = RNNModel(rnn_layer, vocab_size=len(vocab))
net = net.to(device)
d2l.predict_ch8('time traveller', 10, net, vocab, device)

很明显，这种模型根本不能输出好的结果。接下来，我们使⽤ 8.5节中定义的超参数调⽤train_ch8，并且使
⽤⾼级API训练模型。

In [ ]:
num_epochs, lr = 500, 1
d2l.train_ch8(net, train_iter, vocab, lr, num_epochs, device)

与上⼀节相⽐，由于深度学习框架的⾼级API对代码进⾏了更多的优化，该模型在较短的时间内达到了较低
的困惑度。

⼩结

    • 深度学习框架的⾼级API提供了循环神经⽹络层的实现。
    • ⾼级API的循环神经⽹络层返回⼀个输出和⼀个更新后的隐状态，我们还需要计算整个模型的输出层。
    • 相⽐从零开始实现的循环神经⽹络，使⽤⾼级API实现可以加速训练。